In [2]:
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt
from copy import deepcopy
import time
from scipy.signal import argrelmax
import scipy.special as sp
import scipy.sparse as sparse
import sys
sys.path.append("../src") 

from Cubic_Newton import Cubic_Newton_NDE

font = {'weight' : 'bold',
        'size'   : 14}

In [7]:

### Simulation: Generate data from beta mixture distribution with M = 5
### For m = 1,...,M
### b_m (x) = ( Gamma(M+1)/ (Gamma(m)*Gamma(M-m+1)) ) * x^{m-1} * (1-x)^{M-m}

n = 30000
N = n
M0 = 5

weights = np.array([0.35, 0.35, 0.1, 0.1, 0.1])

mixture_idx = np.random.choice(M0, size=n, replace=True, p=weights) + 1
samples = np.fromiter((ss.beta.rvs(m, M0-m+1, size=1) for m in mixture_idx),
                   dtype=np.float64)


# Theoretical PDF plotting -- generate the x and y plotting positions
xs = np.linspace(0, 1, 500)
ys = np.zeros_like(xs)

for m_ in range(M0):
    m = m_ + 1
    ys += ss.beta.pdf(xs, a = m, b= M0-m+1) * weights[m_]
    
    
plt.figure(figsize=(10,8))
plt.rc('font', **font)
plt.plot(xs, ys, linewidth = 3, color='black', label='True Distribution')
plt.hist(samples, density=True, bins=300, color='orange', label='Sample Distribution')
plt.xlabel("x")
plt.ylabel("density")
plt.legend()
plt.show()

In [8]:
# Formulate B
M = 100
rg_M = [m+1 for m in range(M)]
B = np.array([ ss.beta.pdf(samples, m, M-m+1) for m in rg_M ]).transpose()


In [9]:
## use cubic Newton

M_vertices = M
lam0 = (np.ones(M_vertices)/M_vertices) 
S = np.ones(M_vertices)

w, time_cubic, obj_cubic, FW_iters, lam, S = Cubic_Newton_NDE(Bk = B, 
                                                              shape = "decreasing", 
                                                              S = S, 
                                                              lam = lam0, 
                                                              c = 1,
                                                              max_iter = 50, 
                                                              FW_max_iter = 2000, 
                                                              tol = 2e-8,
                                                              short_steps = 5,
                                                              logout = True)


# np.savetxt("Output_data_concave/M=%d_N=%d_our_time" % (M, N), time_cubic)
# np.savetxt("Output_data_concave/M=%d_N=%d_our_obj" % (M, N), obj_cubic)
# gap_cubic = (obj_cubic - obj_Mosek)/(max(np.abs(obj_Mosek), 1))
# print("gap_cubic=", gap_cubic)


----------------------------------
k= 0  obj= -0.0015439081439615756 FW_iters= 241
----------------------------------
k= 1  obj= -0.035753914955711605 FW_iters= 218
----------------------------------
k= 2  obj= -0.05724113431461042 FW_iters= 165
----------------------------------
k= 3  obj= -0.07169175163127416 FW_iters= 135
----------------------------------
k= 4  obj= -0.08173017831896881 FW_iters= 786
----------------------------------
k= 5  obj= -0.08867411823470224 FW_iters= 370
----------------------------------
k= 6  obj= -0.09628417297229275 FW_iters= 153
----------------------------------
k= 7  obj= -0.09921307065782667 FW_iters= 437
----------------------------------
k= 8  obj= -0.09999489454337408 FW_iters= 152
----------------------------------
k= 9  obj= -0.1001272018755222 FW_iters= 262
----------------------------------
k= 10  obj= -0.10013845613326945 FW_iters= 507
----------------------------------
k= 11  obj= -0.10014141973309355 FW_iters= 913
------------------------

In [10]:

plt.figure(figsize=(10,8))
plt.rc('font', **font)
plt.plot(xs, ys, linewidth = 3, color='black', label='True Distribution')
plt.hist(samples, density=True, bins=300, color='orange', label='Sample Distribution')
g = B.dot(w)
X = samples
plt.plot(X[np.argsort(X)], g[np.argsort(X)], color='blue', linewidth = 3, label='Estimated Distribution')



plt.xlabel("x")
plt.ylabel("density")
plt.legend()
plt.show()
